In [21]:
import numpy as np

# mne imports
import mne
from mne import io, read_epochs, concatenate_epochs
from mne.datasets import sample
from mne import Epochs, create_info, events_from_annotations
from mne.datasets import eegbci
from mne.decoding import CSP
from mne.time_frequency import AverageTFR

# EEGNet-specific imports
from EEGModels import EEGNet
from tensorflow.keras import utils as np_utils
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as K
import tensorflow as tf

# PyRiemann imports
from pyriemann.estimation import XdawnCovariances
from pyriemann.tangentspace import TangentSpace
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
# tools for plotting confusion matrices
from matplotlib import pyplot as plt

In [22]:
cd epochs

[Errno 2] No such file or directory: 'epochs'
/Users/jang-wonjun/Documents/eeg/ml/epochs


In [39]:
ls

epoch0-epo.fif    epoch3-epo.fif    epoch7-epo.fif    testset0-epo.fif
epoch1-epo.fif    epoch4-epo.fif    epoch8-epo.fif    testset1-epo.fif
epoch10-epo.fif   epoch5-epo.fif    epoch9-epo.fif    testset2-epo.fif
epoch2-epo.fif    epoch6-epo.fif    testset-epo.fif   testset3-epo.fif


In [40]:
epoch_list = [f'testset{i}-epo.fif' for i in range(4)]

In [41]:
epoch = concatenate_epochs([read_epochs(epoch_i) for epoch_i in epoch_list])
epoch

Reading /Users/jang-wonjun/Documents/eeg/ml/epochs/testset0-epo.fif ...
    Found the data of interest:
        t =   -2000.00 ...    2000.00 ms
        0 CTF compensation matrices available
Not setting metadata
7 matching events found
No baseline correction applied
0 projection items activated
Reading /Users/jang-wonjun/Documents/eeg/ml/epochs/testset1-epo.fif ...
    Found the data of interest:
        t =   -2000.00 ...    2000.00 ms
        0 CTF compensation matrices available
Not setting metadata
100 matching events found
No baseline correction applied
0 projection items activated
Reading /Users/jang-wonjun/Documents/eeg/ml/epochs/testset2-epo.fif ...
    Found the data of interest:
        t =   -2000.00 ...    2000.00 ms
        0 CTF compensation matrices available
Not setting metadata
100 matching events found
No baseline correction applied
0 projection items activated
Reading /Users/jang-wonjun/Documents/eeg/ml/epochs/testset3-epo.fif ...
    Found the data of interest:
    

Number of events,307
Events,1: 622: 623: 614: 615: 61
Time range,-2.000 – 2.000 s
Baseline,off


In [42]:
fmin, fmax =2 ,8
#fmin,fmax=None,None
epoch_filter = epoch.copy().filter(fmin,fmax)

Setting up band-pass filter from 2 - 8 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 2.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 1.00 Hz)
- Upper passband edge: 8.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 9.00 Hz)
- Filter length: 213 samples (1.664 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 2149 out of 2149 | elapsed:    0.2s finished


In [58]:
w_tmin, w_tmax = 0, 0.5


on_event = epoch_filter.copy().crop(w_tmin,w_tmax)
off_event = epoch_filter.copy().crop(-2+w_tmin,-2+w_tmax)
        
X = np.concatenate([on_event.get_data(),off_event.get_data()],axis=0)*1000
y = np.array([1 for i in range(X.shape[0]//2)]+[0 for i in range(X.shape[0]//2)])

In [60]:
#shuffle

indices = np.arange(len(X))
np.random.shuffle(indices)
X = X[indices]
y = y[indices]

In [61]:
X.shape # (trials,channels,samples)

(614, 7, 65)

In [62]:
kernels, chans, samples = 1, 7, 65

In [63]:
y = np_utils.to_categorical(y)

In [65]:
model = tf.keras.models.load_model('../models/EEGnet_model_0.95')

In [66]:
probs       = model.predict(X)
preds       = probs.argmax(axis = -1)  
acc         = np.mean(preds == y.argmax(axis=-1))
print("Classification accuracy: %f " % (acc))

20/20 [==============================] - 0s 1ms/step
Classification accuracy: 0.956026 


In [67]:
preds

array([1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0,
       1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0,
       1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0,
       1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1,
       1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0,
       1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1,
       0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0,
       0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0,

In [68]:
from sklearn.metrics import precision_score
score = precision_score(preds,y.argmax(axis=-1))
print(f"precision : {score*100}%")

precision : 99.0228013029316%
